In [1]:
experiment = "A.indica"
input_genome_name = "GCA_022749755.1_ASM2274975v1_genomic.fna"

experiment_dir = "../experiment"

input_genome_path = f'{experiment_dir}/{experiment}/{input_genome_name}'

temp_path = f"{experiment_dir}/{experiment}/Temp"
result_path = f"{experiment_dir}/{experiment}/Result"

In [2]:
import json
import time
import pickle
from subprocess import Popen, PIPE, STDOUT
import math
import numpy as np
import pandas as pd
import hashlib
import requests
import os, sys, subprocess
from tqdm.contrib.concurrent import process_map
from tqdm.notebook import tqdm
tqdm.pandas()
import multiprocessing as mp
import shutil
import urllib.parse
import glob
import os
import sys
import networkx
from networkx.algorithms.clique import find_cliques as maximal_cliques
from ast import literal_eval
from keras.models import load_model
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
sys.path.append("../src/")
from ct_analizer import get_row
from filter import filter_run
from postprocess import postprocess
from convertor import convert
from preprocessing import get_target, preprocessing

# no filter

## BOI

In [3]:
mfold = pd.read_csv(f"{result_path}/ct_analizer_clustered.csv")
veinna = pd.read_csv(f"{result_path}/viennarna_ct_analizer_clustered.csv")
contrafold  = pd.read_csv(f"{result_path}/contrafold_ct_analizer_clustered.csv")
mxfold2 = pd.read_csv(f"{result_path}/mxfold2_ct_analizer_clustered.csv")

<ipython-input-3-d4316a3c025f>:1: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  mfold = pd.read_csv(f"{result_path}/ct_analizer_clustered.csv")


In [4]:
s_mfold = set(mfold['boi seq'] + mfold["boi dotbracket"])
s_veinna = set(veinna['boi seq'] + veinna["boi dotbracket"])
s_contrafold = set(contrafold['boi seq'] + contrafold["boi dotbracket"])
s_mxfold2 = set(mxfold2['boi seq'] + mxfold2["boi dotbracket"])

In [5]:
print(f'mfold: {len(s_mfold)}')
print(f'veinna: {len(s_veinna)}')
print(f'contrafold: {len(s_contrafold)}')
print(f'mxfold2: {len(s_mxfold2)}')

print(f'mfold & veinna: {len((s_mfold).intersection(s_veinna))}')
print(f'mfold & contrafold: {len((s_mfold).intersection(s_contrafold))}')
print(f'mfold & mxfold2: {len((s_mfold).intersection(s_mxfold2))}')
print(f'veinna & contrafold: {len((s_veinna).intersection(s_contrafold))}')
print(f'veinna & mxfold2: {len((s_veinna).intersection(s_mxfold2))}')
print(f'contrafold & mxfold2: {len((s_contrafold).intersection(s_mxfold2))}')


print(f'veinna & contrafold & mxfold2: {len((s_veinna).intersection(s_contrafold).intersection(s_mxfold2))}')
print(f'mfold & contrafold & mxfold2: {len((s_mfold).intersection(s_contrafold).intersection(s_mxfold2))}')
print(f'mfold & veinna & mxfold2: {len((s_mfold).intersection(s_veinna).intersection(s_mxfold2))}')
print(f'mfold & veinna & contrafold : {len((s_mfold).intersection(s_veinna).intersection(s_contrafold))}')

print(f'mfold & veinna & contrafold & mxfold2: {len((s_mfold).intersection(s_veinna).intersection(s_contrafold).intersection(s_mxfold2))}')

mfold: 24326
veinna: 3715
contrafold: 3036
mxfold2: 971
mfold & veinna: 366
mfold & contrafold: 93
mfold & mxfold2: 0
veinna & contrafold: 81
veinna & mxfold2: 0
contrafold & mxfold2: 1
veinna & contrafold & mxfold2: 0
mfold & contrafold & mxfold2: 0
mfold & veinna & mxfold2: 0
mfold & veinna & contrafold : 44
mfold & veinna & contrafold & mxfold2: 0


## precursor

In [6]:
s_mfold = set(mfold['precursor seq'] + mfold["precursor dotbracket"])
s_veinna = set(veinna['precursor seq'] + veinna["precursor dotbracket"])
s_contrafold = set(contrafold['precursor seq'] + contrafold["precursor dotbracket"])
s_mxfold2 = set(mxfold2['precursor seq'] + mxfold2["precursor dotbracket"])

In [7]:
print(f'mfold: {len(s_mfold)}')
print(f'veinna: {len(s_veinna)}')
print(f'contrafold: {len(s_contrafold)}')
print(f'mxfold2: {len(s_mxfold2)}')

print(f'mfold & veinna: {len((s_mfold).intersection(s_veinna))}')
print(f'mfold & contrafold: {len((s_mfold).intersection(s_contrafold))}')
print(f'mfold & mxfold2: {len((s_mfold).intersection(s_mxfold2))}')
print(f'veinna & contrafold: {len((s_veinna).intersection(s_contrafold))}')
print(f'veinna & mxfold2: {len((s_veinna).intersection(s_mxfold2))}')
print(f'contrafold & mxfold2: {len((s_contrafold).intersection(s_mxfold2))}')


print(f'veinna & contrafold & mxfold2: {len((s_veinna).intersection(s_contrafold).intersection(s_mxfold2))}')
print(f'mfold & contrafold & mxfold2: {len((s_mfold).intersection(s_contrafold).intersection(s_mxfold2))}')
print(f'mfold & veinna & mxfold2: {len((s_mfold).intersection(s_veinna).intersection(s_mxfold2))}')
print(f'mfold & veinna & contrafold : {len((s_mfold).intersection(s_veinna).intersection(s_contrafold))}')

print(f'mfold & veinna & contrafold & mxfold2: {len((s_mfold).intersection(s_veinna).intersection(s_contrafold).intersection(s_mxfold2))}')

mfold: 15427
veinna: 3645
contrafold: 2993
mxfold2: 944
mfold & veinna: 723
mfold & contrafold: 304
mfold & mxfold2: 0
veinna & contrafold: 340
veinna & mxfold2: 3
contrafold & mxfold2: 2
veinna & contrafold & mxfold2: 0
mfold & contrafold & mxfold2: 0
mfold & veinna & mxfold2: 0
mfold & veinna & contrafold : 219
mfold & veinna & contrafold & mxfold2: 0


# Final

In [8]:
mfold = pd.read_csv(f"{result_path}/candidates_postprocessed_clustered.csv")
veinna = pd.read_csv(f"{result_path}/viennarna_candidates_postprocessed_clustered.csv")
contrafold  = pd.read_csv(f"{result_path}/contrafold_candidates_postprocessed_clustered.csv")

## BOI

In [9]:
s_mfold = set(mfold['boi seq'] + mfold["boi dotbracket"])
s_veinna = set(veinna['boi seq'] + veinna["boi dotbracket"])
s_contrafold = set(contrafold['boi seq'] + contrafold["boi dotbracket"])
s_mxfold2 = set()

In [10]:
print(f'mfold: {len(s_mfold)}')
print(f'veinna: {len(s_veinna)}')
print(f'contrafold: {len(s_contrafold)}')
print(f'mxfold2: {len(s_mxfold2)}')

print(f'mfold & veinna: {len((s_mfold).intersection(s_veinna))}')
print(f'mfold & contrafold: {len((s_mfold).intersection(s_contrafold))}')
print(f'mfold & mxfold2: {len((s_mfold).intersection(s_mxfold2))}')
print(f'veinna & contrafold: {len((s_veinna).intersection(s_contrafold))}')
print(f'veinna & mxfold2: {len((s_veinna).intersection(s_mxfold2))}')
print(f'contrafold & mxfold2: {len((s_contrafold).intersection(s_mxfold2))}')


print(f'veinna & contrafold & mxfold2: {len((s_veinna).intersection(s_contrafold).intersection(s_mxfold2))}')
print(f'mfold & contrafold & mxfold2: {len((s_mfold).intersection(s_contrafold).intersection(s_mxfold2))}')
print(f'mfold & veinna & mxfold2: {len((s_mfold).intersection(s_veinna).intersection(s_mxfold2))}')
print(f'mfold & veinna & contrafold : {len((s_mfold).intersection(s_veinna).intersection(s_contrafold))}')

print(f'mfold & veinna & contrafold & mxfold2: {len((s_mfold).intersection(s_veinna).intersection(s_contrafold).intersection(s_mxfold2))}')

mfold: 810
veinna: 304
contrafold: 140
mxfold2: 0
mfold & veinna: 16
mfold & contrafold: 3
mfold & mxfold2: 0
veinna & contrafold: 3
veinna & mxfold2: 0
contrafold & mxfold2: 0
veinna & contrafold & mxfold2: 0
mfold & contrafold & mxfold2: 0
mfold & veinna & mxfold2: 0
mfold & veinna & contrafold : 2
mfold & veinna & contrafold & mxfold2: 0


## precursor

In [11]:
s_mfold = set(mfold['precursor seq'] + mfold["precursor dotbracket"])
s_veinna = set(veinna['precursor seq'] + veinna["precursor dotbracket"])
s_contrafold = set(contrafold['precursor seq'] + contrafold["precursor dotbracket"])
s_mxfold2 = set()

In [12]:
print(f'mfold: {len(s_mfold)}')
print(f'veinna: {len(s_veinna)}')
print(f'contrafold: {len(s_contrafold)}')
print(f'mxfold2: {len(s_mxfold2)}')

print(f'mfold & veinna: {len((s_mfold).intersection(s_veinna))}')
print(f'mfold & contrafold: {len((s_mfold).intersection(s_contrafold))}')
print(f'mfold & mxfold2: {len((s_mfold).intersection(s_mxfold2))}')
print(f'veinna & contrafold: {len((s_veinna).intersection(s_contrafold))}')
print(f'veinna & mxfold2: {len((s_veinna).intersection(s_mxfold2))}')
print(f'contrafold & mxfold2: {len((s_contrafold).intersection(s_mxfold2))}')


print(f'veinna & contrafold & mxfold2: {len((s_veinna).intersection(s_contrafold).intersection(s_mxfold2))}')
print(f'mfold & contrafold & mxfold2: {len((s_mfold).intersection(s_contrafold).intersection(s_mxfold2))}')
print(f'mfold & veinna & mxfold2: {len((s_mfold).intersection(s_veinna).intersection(s_mxfold2))}')
print(f'mfold & veinna & contrafold : {len((s_mfold).intersection(s_veinna).intersection(s_contrafold))}')

print(f'mfold & veinna & contrafold & mxfold2: {len((s_mfold).intersection(s_veinna).intersection(s_contrafold).intersection(s_mxfold2))}')

mfold: 335
veinna: 304
contrafold: 140
mxfold2: 0
mfold & veinna: 71
mfold & contrafold: 16
mfold & mxfold2: 0
veinna & contrafold: 21
veinna & mxfold2: 0
contrafold & mxfold2: 0
veinna & contrafold & mxfold2: 0
mfold & contrafold & mxfold2: 0
mfold & veinna & mxfold2: 0
mfold & veinna & contrafold : 10
mfold & veinna & contrafold & mxfold2: 0
